# SVM
### Homework for Statistical Methods in Data Mining

In [19]:
# generate the 2 classes data including the 4-dimentional data and 10-dimentional data
import numpy as np

# generate data without noise
mu, sigma = 0, 1
train_1_pure = np.random.normal(mu, sigma, (100, 4))
train_0_pure = np.zeros((100, 4))
count = 0
while count < 100:
    temp_data = np.random.normal(mu, sigma, (1, 4))
    if temp_data.dot(np.transpose(temp_data)) > 9 and \
        temp_data.dot(np.transpose(temp_data)) < 16:
        train_0_pure[count, :] = temp_data
        count += 1

# generate data with Gaussian noise ~ N(0,1)
mu, sigma = 0, 1
train_1_noise = np.random.normal(mu, sigma, (100, 10))
train_0_noise = np.zeros((100, 10))
count = 0
while count < 100:
    temp_data = np.random.normal(mu, sigma, (1, 10))
    if np.sum(temp_data[:4]**2) < 16 and np.sum(temp_data[:4]**2) > 9:
        train_0_noise[count, :] = temp_data
        count += 1

print train_1_pure.shape, train_0_pure.shape, train_1_noise.shape, train_0_noise.shape

# generate 1000 test data without noise 
# which includes 500 data labeled 1 and 500 data labeled 0
test_1_pure = np.random.normal(mu, sigma, (500, 4))
test_0_pure = np.zeros((500, 4))
count = 0
while count < 500:
    temp_data = np.random.normal(mu, sigma, (1, 4))
    if temp_data.dot(np.transpose(temp_data)) > 9 and \
        temp_data.dot(np.transpose(temp_data)) < 16:
        test_0_pure[count, :] = temp_data
        count += 1
test_data_pure = np.vstack((test_1_pure, test_0_pure))
test_label_pure = np.ones(1000)
test_label_pure[500:] *= 0

# generate 1000 test data with noise 
# which includes 500 data labeled 1 and 500 data labeled 0
test_1_noise = test_1_pure = np.random.normal(mu, sigma, (500, 10))
test_0_noise = np.zeros((500, 10))
count = 0
while count < 500:
    temp_data = np.random.normal(mu, sigma, (1, 10))
    if np.sum(temp_data[:4]**2) < 16 and np.sum(temp_data[:4]**2) > 9:
        test_0_noise[count, :] = temp_data
        count += 1
test_data_noise = np.vstack((test_1_noise, test_0_noise))
test_label_noise = np.ones(1000)
test_label_noise[500:] *= 0

(100, 4) (100, 4) (100, 10) (100, 10)


In [25]:
from sklearn import svm
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import scipy

train_data_pure = np.vstack((train_1_pure, train_0_pure))
train_label_pure = np.ones(200)
train_label_pure[100:] *= 0

svm_pure = svm.SVC(kernel='rbf')

# specify parameters and distributions to sample from
param_dist = {'C': scipy.stats.expon(scale=100), 'gamma': scipy.stats.expon(scale=.1),
  'kernel': ['rbf'], 'class_weight':['balanced', None]}

# run randomized search with cross-validation
n_iter_search = 1000
random_search = RandomizedSearchCV(svm_pure, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=5)
random_search.fit(train_data_pure, train_label_pure)

print "Best parameters set found on development set:"
print ""
print random_search.best_params_
print ""
print "Best estimator's score:"
print ""
print random_search.best_score_
print ""

# test the result of parameter searching 
print "Result for Test data without noise"
pure_pred = random_search.predict(test_data_pure)
print classification_report(test_label_pure, pure_pred)
print ""



Best parameters set found on development set:

{'kernel': 'rbf', 'C': 536.91083443906507, 'gamma': 0.003891124657703934, 'class_weight': 'balanced'}

Best estimator's score:

0.975

Result for Test data without noise
             precision    recall  f1-score   support

        0.0       0.89      0.98      0.93       500
        1.0       0.98      0.88      0.93       500

avg / total       0.94      0.93      0.93      1000




In [46]:
train_data_noise = np.vstack((train_1_noise, train_0_noise))
train_label_noise = np.ones(200)
train_label_noise[100:] *= 0

svm_noise = svm.SVC(kernel='rbf')
# using default parameter to fit the SVM model
print "Default SVM model:"
print ""
default_model = svm_noise.fit(train_data_noise, train_label_noise)
print default_model
print "Precision in Train dataset"
print default_model.score(train_data_noise, train_label_noise)
print "Precision in Test dataset"
print default_model.score(test_data_noise, test_label_noise)
print "\n"
# specify parameters and distributions to sample from
param_dist = {'C': scipy.stats.uniform(loc=0.5,scale=5), 
              'gamma': scipy.stats.uniform(loc=0.01, scale=5),
              'kernel': ['rbf']}

# run randomized search with cross-validation
n_iter_search = 1000
random_search_noise = RandomizedSearchCV(svm_noise, param_distributions=param_dist,
                                         n_iter=n_iter_search, cv=5)
random_search_noise.fit(train_data_noise, train_label_noise)

print "Best parameters set found on development set:"
print ""
print random_search_noise.best_params_
print ""
print "Best estimator's score:"
print ""
print random_search_noise.best_score_
print ""

# test the result of parameter searching 
print "Result for Test data with noise"
noise_pred = random_search_noise.predict(test_data_noise)
print classification_report(test_label_noise, noise_pred)
print ""


Default SVM model:

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Precision in Train dataset
0.875
Precision in Test dataset
0.589


Best parameters set found on development set:

{'kernel': 'rbf', 'C': 4.6823712812494005, 'gamma': 0.7181712920337191}

Best estimator's score:

0.625

Result for Test data with noise
             precision    recall  f1-score   support

        0.0       0.60      0.77      0.68       500
        1.0       0.69      0.49      0.57       500

avg / total       0.64      0.63      0.63      1000




## Specification for Part 2
From the result showing in the above, we find that even if using RBF kernel, the SVM classifier meets the problem of overfitting or the curse of dimention. 

Once adding the extra useless features to the data, we could see the much worse results of the test data given by the SVM classifier using RBF kernel. Howervr, what makes me at a loss is that by searching the parameter the best estimator's score is only 0.625 in dataset of training. What's more, the default model of the SVC's score is 0.875 in data set of training. After reflecting on the problem, I found the default model's score in dataset of testing is so low. Finally, I think it's because we use the whole dataset to train the default model but cross-validation methods to find the best parameter.  

Back to the curse of dimention, the svm classifier using RBF kernel has advantage of controling the feature number use in the model by modifying the parameter gamma. However, we couldn't prohibit it from overfitting or meet the curse of dimention, once too much noise has been added into the feature space.

In [44]:
# SVM using poly kernel

poly_pure = svm.SVC(kernel='poly', degree=8).fit(train_data_pure, train_label_pure)
poly_noise = svm.SVC(kernel='poly', degree=6).fit(train_data_noise, train_label_noise)

print 'SVM using poly kernel:'
print ''
print "Test score of data without noise:" 
print poly_pure.score(test_data_pure, test_label_pure)
print ''
print 'Test score of data with noise:'
print poly_noise.score(test_data_noise, test_label_noise)
print ''


SVM using poly kernel:

Test score of data without noise:
0.915

Test score of data with noise:
0.52



## Specification for Part 3
Using the default parameter, we find the SVM classifier which uses liner kernel doesn't surpass the SVM model using the RBF kernel. Also the liner kernel will meet the problem of overfitting or curse of dimention.